多标签车牌识别

一些参考:
https://discuss.pytorch.org/t/multi-label-classification-in-pytorch/905/16
Multi Label Classification in pytorch

https://discuss.pytorch.org/t/calculating-accuracy-for-a-multi-label-classification-problem/2303
Calculating accuracy for a multi-label classification problem

https://discuss.pytorch.org/t/equivalent-of-tensorflows-sigmoid-cross-entropy-with-logits-in-pytorch/1985
Equivalent of TensorFlow’s Sigmoid Cross Entropy With Logits in Pytorch

https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning
Starting Kit for PyTorch Deep Learning

http://stackoverflow.com/questions/34240703/difference-between-tensorflow-tf-nn-softmax-and-tf-nn-softmax-cross-entropy-with
difference between tensorflow tf.nn.softmax and tf.nn.softmax_cross_entropy_with_logits



In [20]:
import os
import os.path
import random
import cv2
import math
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as torch_utils_data

In [21]:
DIGITS = "0123456789"
LETTERS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
#PROVINCE="黑吉辽京津内冀鲁豫徽苏沪浙赣闽粤鄂湘云贵川渝藏青宁新陕甘宁晋" #30
CHARS = LETTERS + DIGITS
NPLEN=7
NUM_CLASSES=len(CHARS)*NPLEN

In [22]:
class anprmodel(nn.Module):
    def __init__(self):
        super(anprmodel,self).__init__()
        self.num_classes=NUM_CLASSES
        self.conv1=nn.Conv2d(1,48,kernel_size=5,padding=2)
        self.pool1=nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2=nn.Conv2d(48,64,kernel_size=5,padding=2)
        self.pool2=nn.MaxPool2d(kernel_size=(2,1),stride=(2,1)) 
        self.conv3=nn.Conv2d(64,128,kernel_size=5,padding=2)
        self.pool3=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))        
        self.fc1=nn.Linear(32*8*128,2048)
        self.fc2=nn.Linear(2048,NUM_CLASSES)

    def forward(self,x): 
        x=F.relu(self.pool1(self.conv1(x)))  #128*64
        x=F.relu(self.pool2(self.conv2(x)))  #64*32
        x=F.relu(self.pool3(self.conv3(x)))  #64*16
        x=x.view(-1,32*8*128)                        #32*8
        x=F.relu(self.fc1(x))
        x=self.fc2(x)                                       
        #return x
        return F.sigmoid(x)

In [23]:
class NPSET(torch_utils_data.Dataset):
    picroot='np'
   
    def code_to_vec(self,code):  #(self,p, code):
        def char_to_vec(c):
            y = np.zeros((len(CHARS),),dtype=np.float)   #multilabel soft margin loss
            #y = np.zeros((len(CHARS),),dtype=np.long)   #multilabel margin loss
            y[CHARS.index(c)] = 1.0
            return y
        c = np.vstack([char_to_vec(c) for c in code])
        return c.flatten()
        #return np.concatenate([[1. if p else 0], c.flatten()])

    def __getitem__(self,index):
        label=self.labels[index]
        im=cv2.imread(self.dataset[index],cv2.IMREAD_GRAYSCALE)
        img=np.reshape(im,(1,im.shape[0],im.shape[1]))
        if self.data_transform is not None:
            img=self.data_transform(img)
        labelarray=self.code_to_vec(label)
        return img,torch.FloatTensor(labelarray)
        #return img,torch.LongTensor(labelarray)

    def __len__(self):
        return self.len

    def __init__(self,root,data_transform=None):
        self.picroot=root
        self.data_transform=data_transform
        self.labels=[]
        self.dataset=[]
        if not os.path.exists(self.picroot):
            raise RuntimeError('{} doesnot exists'.format(self.picroot))
        for root,dnames,filenames in os.walk(self.picroot):
            for filename in filenames:
                picfilename=os.path.join(self.picroot,filename)  #file name:
                m=filename.split('_')  #filename style: xxxxxxxx_xxxxxxx_x.png
                self.labels.append(m[1])
                self.dataset.append(picfilename)
            self.len=len(filenames)

In [24]:
#104.149524851 3266.80839036
model=anprmodel()
#model.features=torch.nn.DataParallel(model.features)
#model.cuda()
cudnn.benchmark=True
batch_size=5
data_transform=transforms.Compose([transforms.ToTensor(),
                                   #transforms.RandomHorizontalFlip(),
                                   transforms.Normalize((104.149524851,), (3266.80839036,)),
                             ])
npset = NPSET(root='/home/wang/git/nppic/su_np-128x64/train', data_transform=data_transform)
nploader = torch.utils.data.DataLoader(npset, batch_size=batch_size, shuffle=True, num_workers=1)  #train
criterion=nn.MultiLabelSoftMarginLoss()  #MultiLabelMarginLoss()
#criterion=F.binary_cross_entropy()
optimizer=torch.optim.SGD(model.parameters(),0.001,momentum=0.9)
#optimizer=torch.optim.Adam(model.parameters())

In [25]:
npvalset=NPSET(root='/home/wang/git/nppic/su_np-128x64/val', data_transform=data_transform)
npvalloader=torch.utils.data.DataLoader(npvalset, batch_size=10, shuffle=False, num_workers=1) #validate

In [26]:
for epoch in range(0,30):
    #Sets the learning rate to the initial LR decayed by 10 every 30 epochs
    #lr=0.1*(0.1**(epoch//30))
    #for param_group in optimizer.param_groups:
    #    param_group['lr']=lr
    #train
    model.train()
    for i,data in enumerate(nploader):
        inputs,targets = data
        input_var=torch.autograd.Variable(inputs)
        target_var=torch.autograd.Variable(targets)
        
        optimizer.zero_grad()
        output=model(input_var)
        #porcess loss
        #loss=criterion(output,target_var)
        loss = F.binary_cross_entropy(output, target_var)
        loss.backward()
        optimizer.step()
        #
        if (i+1)% 20 == 0:
             print('Train Epoch: {} [{}/{} ({:.0f}%)]\\tLoss: {:.6f}'.format(
                  epoch, i * len(data), len(nploader.dataset),
                  100. * i / len(nploader), loss.data[0]))

Train Epoch: 0 [38/500 (19%)]\tLoss: 0.692813
Train Epoch: 0 [78/500 (39%)]\tLoss: 0.692181
Train Epoch: 0 [118/500 (59%)]\tLoss: 0.691722
Train Epoch: 0 [158/500 (79%)]\tLoss: 0.691328
Train Epoch: 0 [198/500 (99%)]\tLoss: 0.690720
Train Epoch: 1 [38/500 (19%)]\tLoss: 0.689959
Train Epoch: 1 [78/500 (39%)]\tLoss: 0.689285
Train Epoch: 1 [118/500 (59%)]\tLoss: 0.688647
Train Epoch: 1 [158/500 (79%)]\tLoss: 0.687790
Train Epoch: 1 [198/500 (99%)]\tLoss: 0.687011
Train Epoch: 2 [38/500 (19%)]\tLoss: 0.685873
Train Epoch: 2 [78/500 (39%)]\tLoss: 0.684709
Train Epoch: 2 [118/500 (59%)]\tLoss: 0.683183
Train Epoch: 2 [158/500 (79%)]\tLoss: 0.681328
Train Epoch: 2 [198/500 (99%)]\tLoss: 0.678924
Train Epoch: 3 [38/500 (19%)]\tLoss: 0.675632
Train Epoch: 3 [78/500 (39%)]\tLoss: 0.671177
Train Epoch: 3 [118/500 (59%)]\tLoss: 0.665022
Train Epoch: 3 [158/500 (79%)]\tLoss: 0.655608
Train Epoch: 3 [198/500 (99%)]\tLoss: 0.639786
Train Epoch: 4 [38/500 (19%)]\tLoss: 0.612946
Train Epoch: 4 [78/500

In [27]:
print targets.size()
t=torch.Tensor(targets.size())
t.copy_(targets)
targetre=t.resize_(5,7,36)
print targetre[0]

torch.Size([5, 252])


Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0

In [28]:
npvaliter=iter(npvalloader)
inputs,targets=npvaliter.next()
input_var = torch.autograd.Variable(inputs, volatile=True)

In [29]:
output = model(input_var)

In [33]:
#type(output)
#o=torch.sigmoid(output).data
print output.data
#print o


 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
          ...             ⋱             ...          
 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
 0.0006  0.0006  0.0006  ...   0.0891  0.0742  0.1139
[torch.FloatTensor of size 10x252]



In [31]:
recs=[]
for _,r in enumerate(o):
    rec=[]
    for i,re in enumerate(r):
        if re>0.5:
            rec.append(i)
    recs.append(rec)        
print recs

#print output.size()
#print output
#print targets

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

In [ ]:
o=torch.sigmoid(output).data

In [ ]:
outputdata=output.data
t=torch.Tensor(outputdata.size())
t.copy_(outputdata)
outputre=t.resize_(4,7,36)
t.copy_(targets)
targetre=t.resize_(4,7,36)

In [ ]:
print outputdata.size()
print outputre.size()

In [ ]:

_,predicted = torch.max(outputre, 2)
print predicted

In [ ]:
print len(nploader) 
print len(inputs)
print len(nploader.dataset)

In [ ]:
    #validate
    model.eval()
    for i, data in enumerate(npvalloader):
        (inputs, target)=data
        #target = target.cuda()
        input_var = torch.autograd.Variable(inputs, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        # compute output
        output = model(input_var)
        #porcess loss

In [ ]:
#将su_np-256x128,541张仅车牌的灰度图像，调整大小输出到su_np-128x64中
SOURCE='/home/wang/git/nppic/su_np-256x128'
DEST='/home/wang/git/nppic/su_np-128x64'
for parent,dirnames,filenames in os.walk(SOURCE):
    for fname in filenames:
        im=cv2.imread(os.path.join(parent,fname))
        imr=cv2.resize(im,(128,64))
        img=cv2.cvtColor(imr,cv2.COLOR_BGR2GRAY)
        #cv2.imwrite(os.path.join(DEST,fname),img)
print 'over'        

In [ ]:
#计算SOURCE目录下图片的均值和方差
SOURCE='/home/wang/git/nppic/su_np-128x64'
f=[]
for parent,dirnames,filenames in os.walk(SOURCE):
     for i,fname in enumerate(filenames):
        f.append(os.path.join(parent,fname))
img=np.ndarray((len(f),64,128))
for i,fname in enumerate(f):
    im=cv2.imread(fname)
    img[i]=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
mean=np.mean(img)
var=np.var(img)
print mean,var

In [ ]:
def code_to_vec(p, code):
    def char_to_vec(c):
        y = np.zeros((len(CHARS),))
        y[CHARS.index(c)] = 1.0
        return y
    c = np.vstack([char_to_vec(c) for c in code])
    return c,c.flatten()
a,b=code_to_vec('1','0AV253C')
print 'NUMCLASSES={}'.format(NUM_CLASSES)
print a.shape,a
print '-----------------'
print b.shape,b

In [ ]:
#multi label classification example
#---------------------------------------------------------------------------------------------
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from  torch.autograd import Variable

train=[]
labels=[]
for i in range(10000):
    category=(np.random.choice([0,1]),np.random.choice([0,1]))
    if category==(1,0):
        train.append([np.random.uniform(0.1,1),0])
        labels.append([1,0,1])
    if category==(0,1):
        train.append([0,np.random.uniform(0.1,1)])
        labels.append([0,1,0])
    if category==(0,0):
        train.append([np.random.uniform(0.1,1),np.random.uniform(0.1,1)])
        labels.append([0,0,1])
        
class _classifier(nn.Module):
    def __init__(self,nlabel):
        super(_classifier,self).__init__()
        self.main=nn.Sequential(
            nn.Linear(2,64),
            nn.ReLU(),
            nn.Linear(64,nlabel),
        )
    def forward(self,input):
        return self.main(input)
    
nlabel=len(labels[0])    
classifier=_classifier(nlabel)
optimizer=optim.Adam(classifier.parameters())
criterion=nn.MultiLabelSoftMarginLoss()

epochs=1
for epoch in range(epochs):
    losses=[]
    for i,sample in enumerate(train):
        inputv=Variable(torch.FloatTensor(sample)).view(1,-1)
        labelsv=Variable(torch.FloatTensor(labels[i])).view(1,-1)
        output=classifier(inputv)
        loss=criterion(output,labelsv)
        print loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning


import pandas as pd
from torch import np # Torch wrapper for Numpy

import os
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn.preprocessing import MultiLabelBinarizer

IMG_PATH = '../input/train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = '../input/train.csv'

class KaggleAmazonDataset(Dataset):
    """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['image_name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['image_name']
        self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)
        
transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])

dset_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)
        


train_loader = DataLoader(dset_train,
                          batch_size=256,
                          shuffle=True,
                          num_workers=4 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

model = Net() # On CPU
#model = Net().cuda() # On GPU
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

for epoch in range(1, 2):
    train(epoch)
    


Train Epoch: 1 [0/40479 (0%)]	Loss: 0.690799
Train Epoch: 1 [2560/40479 (6%)]	Loss: 0.686433
Train Epoch: 1 [5120/40479 (13%)]	Loss: 0.681382
Train Epoch: 1 [7680/40479 (19%)]	Loss: 0.673910
Train Epoch: 1 [10240/40479 (25%)]	Loss: 0.667018
Train Epoch: 1 [12800/40479 (31%)]	Loss: 0.656438
Train Epoch: 1 [15360/40479 (38%)]	Loss: 0.645444
Train Epoch: 1 [17920/40479 (44%)]	Loss: 0.628610
Train Epoch: 1 [20480/40479 (50%)]	Loss: 0.600967
Train Epoch: 1 [23040/40479 (57%)]	Loss: 0.570082
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.520596
Train Epoch: 1 [28160/40479 (69%)]	Loss: 0.465080
Train Epoch: 1 [30720/40479 (75%)]	Loss: 0.412709
Train Epoch: 1 [33280/40479 (82%)]	Loss: 0.365693
Train Epoch: 1 [35840/40479 (88%)]	Loss: 0.357215
Train Epoch: 1 [38400/40479 (94%)]	Loss: 0.340456

    

https://discuss.pytorch.org/t/feedback-on-pytorch-for-kaggle-competitions/2252

https://discuss.pytorch.org/t/multi-label-classification-in-pytorch/905/13
@AjayTalati

Either after your last fc you do a sigmoid and then you use BCELoss or F.binary_crossentropy as your criterion/lossfunction

Or you directly use MultiLabelSoftMarginLoss as your loss function (it comes with sigmoid inside)

Now once you have your prediction, you need to threshold. 0.5 is the default naive way but it's probably not optimal. In any case, once you get there, great !

Next part is technical optimization, you can do Multilabel classification without

Regarding the threshold, you might want to optimize either a common threshold for all your outputs (it can be 0.2, 0.5, 0.123456 who knows) or optimize a threshold per label class, especially if your classes as unbalanced.
You will need a solid validation set and a MultiLabel evaluation metrics (Hamming Loss, F1-score, Fbeta score).

An example code for the first strategy is here on Kaggle2.

For the second strategy, I'm deep into various papers myself so I can't help yet.
One thing to keep in mind is your "best threshold" will probably overfit the validation set, so use regularization, cross-validation or other anti-overfitting strategy.

https://discuss.pytorch.org/t/equivalent-of-tensorflows-sigmoid-cross-entropy-with-logits-in-pytorch/1985/11
@AjayTalati I managed to use BCELoss, binary_crossentropy and MultiLabelSoftMarginLoss on a MultiLabel problem

Here is the basic code

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
And the source is here6.

For BCELoss you can use criterion = BCELoss() and then loss = criterion(output, target) but as @Misha_E said, the NN must return a sigmoid activation.